In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge conllu


In [2]:
import stanza
import os
from estnltk import Text
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger
from estnltk.converters.conll_exporter import  sentence_to_conll
import numpy as np

In [3]:
model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
#stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, resources_path=model_path)
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


In [4]:
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from collections import defaultdict
from textwrap import wrap

class graphFunctions:
    
    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    
    #TODO params to **kwargs
    def drawGraph(G, title=None, filename=None, highlight=[]):
        
        # soovitud tipud punaseks
        color_map = ['red' if node in highlight else 'lightskyblue' for node in G]
        
        # joonise suurus, et enamik puudest ära mahuks
        plt.rcParams["figure.figsize"] = (18.5, 10.5)
        
        #pealkiri
        if title:
            title = ("\n".join(wrap( title, 120)))
            plt.title(title)
        
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True, node_color=color_map)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        
        #kui failinimi, siis salvestame faili
        #kui pole, siis joonistame väljundisse
        if filename:
            plt.savefig(f'{filename}.png', dpi=100)
        else:
            plt.show()
        plt.clf()
    
    # conllu lause objektist graafi tegemine
    def make_graph_conllu(sentence):
        G = nx.DiGraph()
        for data in sentence:
            if isinstance(data['id'], int):
                #paneme graafi kokku
                G.add_node(data['id'], id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], form=data['form'])
                G.add_edge(data['id'] - data['id'] + data['head'], data['id'], deprel = data['deprel'])
        return G
    
    # stanza stanza_syntax objektist graafi tegemine
    def make_graph_stanza(sentence):
        G = nx.DiGraph()
        for data in sentence:
            #print (data)
            if isinstance(data['id'], int):
                #paneme graafi kokku
                G.add_node(data['id'], id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], form=data.text)
                G.add_edge(data['id'] - data['id'] + data['head'], data['id'], deprel = data['deprel'])
        return G

    # lyhim tee graafi tippude vahel ning nn reversed kaartega graafist sama
    def get_shortest_paths(G):

        # lyhim tee tippude vahel
        path = nx.all_pairs_shortest_path_length(G)
        path_reversed = nx.all_pairs_shortest_path_length(G.reverse())
        # kauguste maatriksid
        dpath = {x[0]:x[1] for x in path}
        dpath_reversed = {x[0]:x[1] for x in path}
        return {'dict': path,  'dict_reversed': path_reversed, 'matrix': dpath, 'matrix_reversed': dpath_reversed}

    #tagastab array-na syntaksipuu graafi propreteid, tippu 0 ignoreerib
    def get_prop(graph, property_name):
        return [graph.nodes[node][property_name] for node in sorted([node for node in graph.nodes]) if node]
    
    #leiab, millised tipud suuremast graafist1 on puudu graafis2
    def get_nodes_diff(graph1, graph2):
        return [ node for node in graph1 if not node in graph2 ]

#sisendiks on lausepuu nn tabeli kujul connlu sentence
# teeme sellest graafi connections abil
# siis kustutame soovitud depreli haru ja paneme lause tagasi kokku
import networkx as nx

#tekst on tokeniseeritud, võib tühikuga otsida küll
def is_enne_kui_examples(text):
    sent = ' ' + text.lower() + ' '
    kui = [' rohkem kui ', ' vähem kui ', ' samapalju kui ', ' enam kui ']
    for k in kui:
        if k in sent:
            return True
    return False

def remove_deprel_ennekui(inputG, rem_deprel):
    
    #eemaldame ainult siis, kui advmod lemma on [rohkem, vähem, samapalju, enam]
    # ja kui sellele s]nale vahetult järgnes lemma "kui"
    
    G = inputG.copy()
    #originaallause graaf
    G_original = inputG.copy() 
   
    #tipud eemaldatava depreliga
    rem_deprel_nodes = graphFunctions.get_nodes_by_attributes(G, 'deprel', rem_deprel)
    
    # lemmade nimekiri praegu ei kasuta, vb tuleb t2psustada, mis lemmat eemaldame
    # näiteks, antud juhul võibolla tahame eemaldada ainult seda advmodi, kus on kui-le järgnev sõna või kui
    #lemmas = nx.get_node_attributes(G, "lemma")
    
    #eemaldame tipud, mis on otsitava deprel'iga
    for n in rem_deprel_nodes:
        
        #lisatingumus enne_kui jaoks
        if G.nodes[n]['lemma'] in ['rohkem', 'vähem', 'samapalju', 'enam']:
            if n+1 in G.nodes and G.nodes[n+1]['lemma'] == 'kui':
                G.remove_node(n)
    
    # lyhim tee tippude vahel ja kauguste maatriksid
    paths = graphFunctions.get_shortest_paths(G)
    #paths['matrix'] on dictionary lyhima kaugusega seotud tippudega
    #print ('dpath', dpath)
    
    #eemaldame kõik tipud, mis pole enam sidusad
    #eeldame siin praegu, et verbi ei eemaldata
    nodes = [node for node in G.nodes]
    for node in nodes:
        if node>0 and not node in paths['matrix'][0]:
            G.remove_node(node)
    
    
    deprels = [G.nodes[node]['deprel'] if node in G.nodes else '_' for node in sorted([node for node in G_original.nodes]) if node ]
    
    return G


        
def analyze_as_graph(text, stanza_tagger):
    #uus analüüs
    short_sent_txt = Text(text)
    short_sent_txt.analyse('all')
    stanza_tagger.tag(short_sent_txt)
    new_graph = graphFunctions.make_graph_stanza(short_sent_txt.stanza_syntax)
    return new_graph

#add blanks at position of indexes instead of removed words
def add_blanks(a, indexes, blank='_'):
    array=a.copy()
    for ind in sorted(indexes):   
        array.insert(ind-1, blank)
       
    return array

def remove_removed(a, indexes):
    #print (array, indexes)
    array=a.copy()
    for ind in reversed(sorted(indexes)):
        array.pop(ind-1)
    return array


def is_equal(arr1, arr2):
    if not len(arr1) == len(arr2):
        return False
    for i in range(len(arr1)):
        if not arr1[i] == arr2[i]:
            return False
    return True

    

In [5]:
# make list of files
from io import open
from conllu import parse


CONLLU_ROOT='/Users/rabauti/repos/tu/EstUD/vers2_8_enhanced/'
connlu_files = []
for path, subdirs, files in os.walk(CONLLU_ROOT):
    for name in files:
        if name.endswith('.conllu'):
            connlu_files.append((path.replace(CONLLU_ROOT, ''), name))
connlu_files[:3]

[('EDTtreex/Testtreex', 'aja_pm20000218_osa_4_ud28.enhanced.treex.conllu'),
 ('EDTtreex/Testtreex', 'tea_dr8020_ud28.enhanced.treex.conllu'),
 ('EDTtreex/Testtreex', 'aja_pm20000218_osa_3_ud28.enhanced.treex.conllu')]

In [6]:
%%time

import csv

from pathlib import Path
Path("./results").mkdir(parents=True, exist_ok=True) 
Path("./results/imgtrees").mkdir(parents=True, exist_ok=True) 
Path("./results/imgtrees/changed").mkdir(parents=True, exist_ok=True) 
Path("./results/imgtrees/conserved").mkdir(parents=True, exist_ok=True) 
Path("./results/imgtrees/constant").mkdir(parents=True, exist_ok=True) 


stats = {}
stats['faile_kokku'] = len(connlu_files)
stats['lauseid_kokku'] = 0
stats['ennekui_lauseid_kokku'] = 0

stats['ennekui_lauseid_muutunud'] = 0
stats['ennekui_lauseid_konserveerunud'] = 0
stats['ennekui_lauseid_puudub_advmod'] = 0


outputfile_changed = f'results/UD_ennekui_changed.tsv' 
outputfile_conserved = f'results/UD_ennekui_conserved.tsv'
outputfile_constant = f'results/UD_ennekui_constant.tsv' 

outChanged = open(outputfile_changed, 'w', encoding='utf-8')
outConserved = open(outputfile_conserved, 'w', encoding='utf-8')
outConstant = open(outputfile_constant, 'w', encoding='utf-8')

# create the csv writer
wChanged = csv.writer(outChanged, delimiter='\t')
wConserved = csv.writer(outConserved, delimiter='\t')
wConstant = csv.writer(outConstant, delimiter='\t')


wConserved.writerow(('FILE','SENTENCE_ID', 'TYPE', 'TEXT', 'DEPRELS',),)
wChanged.writerow(('FILE','SENTENCE_ID', 'TYPE', 'TEXT', 'DEPRELS',),)
wConstant.writerow(('FILE','SENTENCE_ID', 'TYPE', 'TEXT', 'DEPRELS',),)

emptyrow = ('', '', '', '', '', )

deprel = 'advmod'

for directory, file  in connlu_files:
    
    filepath = f"{directory}/{file}"
    data_file = open(f"{CONLLU_ROOT}{directory}/{file}", "r", encoding="utf-8")

    data = data_file.read()
    sentences = parse(data)
    
    #cols  sent id type  sentence deprels
    changed = []
    conserved = []
    for sent in sentences:
        nodes_diff = []
        stats['lauseid_kokku'] +=1
        
        row = []
        originaltext = sent.metadata['text']
        sent_id = sent.metadata['sent_id']
        global_sent_id = f'{directory}/{file}_{sent_id}'.replace('/', '__')
        
        #print ('O:', originaltext)
        g_orig = graphFunctions.make_graph_conllu(sent)
        deprel_origin = graphFunctions.get_prop(g_orig, 'deprel')
        
        
        #v6tame siit, sest originaalkuju on tokeniseerimata
        original_text = " ".join(graphFunctions.get_prop(g_orig, 'form'))
        
        if  not is_enne_kui_examples(original_text): 
            #print ('NOT', originaltext)
            continue
        stats['ennekui_lauseid_kokku'] +=1
        
        #print ('-----')

       
        g_short = remove_deprel_ennekui(g_orig, deprel)
        short_text = " ".join(graphFunctions.get_prop(g_short, 'form'))
        
        #esimene täht suureks
        

        #kui tekst ei muutunud, jätkame
        if original_text == short_text: 
            
            row1 = [filepath\
                , sent_id\
                , 'O'\
                , original_text\
                , " ".join(deprel_origin)]
            wConstant.writerow(row1)
            wConstant.writerow(emptyrow)
            stats['ennekui_lauseid_puudub_advmod'] +=1
            
            graphFunctions.drawGraph(g_orig, " ".join(graphFunctions.get_prop(g_orig, 'form')), f'results/imgtrees/constant/{global_sent_id}_original', nodes_diff)
       
            continue

        

        #kui tekst muutus, siis:
        #teeme uue lauseanalüüsi Stanzaga
        #teisendame graafiks
        #graafist küsime deprelid

        nodes_diff = graphFunctions.get_nodes_diff(g_orig, g_short)
        if 1 in nodes_diff:
            short_text = short_text[0].upper() + short_text[1:]
     
        #uus analüüs
        new_graph = analyze_as_graph(short_text, stanza_tagger)

        #peame teadma, mis nr sõnad kadusid, et saaks võrrelda??
        
        
        
        deprel_short = graphFunctions.get_prop(new_graph, 'deprel')
        deprel_with_blanks = add_blanks(deprel_short,nodes_diff)
        original_removed = remove_removed(deprel_origin, nodes_diff)
        #graphFunctions.drawGraph(g_short, " ".join(get_prop(g_short, 'form')), f'imgtrees/{global_sent_id}_removed')
 
        row1 = [filepath\
                , sent_id\
                , 'O'\
                , original_text\
                , " ".join(deprel_origin)]
    
        row2 = [filepath\
                , sent_id \
                , 'S'\
                , short_text\
                , " ".join(deprel_with_blanks)]
    
        conserved = is_equal(deprel_short, original_removed)
        if conserved:
            wConserved.writerow(row1)
            wConserved.writerow(row2)
            wConserved.writerow(emptyrow)
            stats['ennekui_lauseid_konserveerunud'] +=1
            graphFunctions.drawGraph(g_orig, " ".join(graphFunctions.get_prop(g_orig, 'form')), f'results/imgtrees/conserved/{global_sent_id}_original', nodes_diff)
       

            

        else:
            wChanged.writerow(row1)
            wChanged.writerow(row2)
            wChanged.writerow(emptyrow)
            graphFunctions.drawGraph(g_orig, " ".join(graphFunctions.get_prop(g_orig, 'form')), f'results/imgtrees/changed/{global_sent_id}_original', nodes_diff)
            graphFunctions.drawGraph(new_graph, " ".join(graphFunctions.get_prop(new_graph, 'form')), f'results/imgtrees/changed/{global_sent_id}_post')
            stats['ennekui_lauseid_muutunud'] += 1
            
outChanged.close()
outConserved.close()
outConstant.close()
    

print (f"Kokku faile:  {stats['faile_kokku']}")
print (f"Kokku lauseid: {stats['lauseid_kokku']}")

print (f"\tLauseid, kus oli nn \"enne-kui\" konstruktsioon: {stats['ennekui_lauseid_kokku']}")

print (f"\t\t Polnud advmod tippu eemaldamiseks: {stats['ennekui_lauseid_puudub_advmod']}")

print (f"\t\t Süntaks konserveerus: {stats['ennekui_lauseid_konserveerunud']}")

print (f"\t\t Süntaks ei konserveerunud: {stats['ennekui_lauseid_muutunud']}")



Kokku faile:  117
Kokku lauseid: 32644
	Lauseid, kus oli nn "enne-kui" konstruktsioon: 130
		 Polnud advmod tippu eemaldamiseks: 35
		 Süntaks konserveerus: 23
		 Süntaks ei konserveerunud: 72
CPU times: user 1min 22s, sys: 9.15 s, total: 1min 31s
Wall time: 2min 13s


<Figure size 1332x756 with 0 Axes>